In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
options = Options() 
driver = webdriver.Chrome(options=options)
driver.get('https://www.clasificadosonline.com/RealEstate.asp')

In [4]:
multi_check_box_toggle = driver.find_element(By.XPATH,'//*[@id="form1"]/div/table[2]/tbody/tr/td/div[1]')

In [5]:
select_all = driver.find_element(By.XPATH,'//*[@id="form1"]/div/table[2]/tbody/tr/td/div[2]/div[1]/input')

In [6]:
ver_listado = driver.find_element(By.XPATH,'//*[@id="BtnSearchListing"]')

In [7]:
multi_check_box_toggle.click()
time.sleep(2)
select_all.click()
time.sleep(2)
ver_listado.click()

In [8]:
order_by = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]')

In [9]:
ultimos_publicados = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]/option[7]')

In [10]:
order_by.click()

In [11]:
ultimos_publicados.click()

In [12]:
list_cards = []

for i in range(1,16):
    table_1 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1)'
    table_1_title = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
    table_1_rooms = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
    table_1_price = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
    table_1_type = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
    table_1_location = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
    table_1_link = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
    
    table_2 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(3)'
    table_2_title = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
    table_2_rooms = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
    table_2_price = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
    table_2_type = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
    table_2_location = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
    table_2_link = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
    
    dictionary_cards = {}
    dictionary_cards_2 = {}
    try:
        table_1_type_2 = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_1_type_2_text = driver.find_element(By.CSS_SELECTOR, table_1_type_2).text
        dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text + ' ' + table_1_type_2_text
        
        table_2_type_2 = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_2_type_2_text = driver.find_element(By.CSS_SELECTOR, table_2_type_2).text
        dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text + ' ' + table_2_type_2_text

    except NoSuchElementException:
        dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text
        dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text
    try:
        table_1_broker = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        dictionary_cards['Broker'] = driver.find_element(By.CSS_SELECTOR, table_1_broker).get_attribute('alt')

        table_2_broker = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        dictionary_cards_2['Broker'] = driver.find_element(By.CSS_SELECTOR, table_2_broker).get_attribute('alt')

    except NoSuchElementException:
        dictionary_cards['Broker'] = 'No Broker'
        dictionary_cards_2['Broker'] = 'No Broker'

    dictionary_cards['Title'] = driver.find_element(By.CSS_SELECTOR, table_1_title).text
    dictionary_cards['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_1_rooms).text
    dictionary_cards['Price'] = driver.find_element(By.CSS_SELECTOR, table_1_price).text
    dictionary_cards['Location'] = driver.find_element(By.CSS_SELECTOR, table_1_location).text
    dictionary_cards['Link'] = driver.find_element(By.CSS_SELECTOR, table_1_link).get_attribute('href')
    list_cards.append(dictionary_cards)

    dictionary_cards_2['Title'] = driver.find_element(By.CSS_SELECTOR, table_2_title).text
    dictionary_cards_2['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_2_rooms).text
    dictionary_cards_2['Price'] = driver.find_element(By.CSS_SELECTOR, table_2_price).text
    dictionary_cards_2['Location'] = driver.find_element(By.CSS_SELECTOR, table_2_location).text
    dictionary_cards_2['Link'] = driver.find_element(By.CSS_SELECTOR, table_2_link).get_attribute('href')
    list_cards.append(dictionary_cards_2)


In [40]:
list_cards
df = pd.json_normalize(list_cards)
df

,Type,Broker,Title,Rooms,Price,Location,Link
0,", Solar",ClasificadosOnline Cacao de House Hunt Real Estate #18152,BO. CACAO,,"$57,999","Barrio-Cacao , Quebradillas",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4767842
1,", Comercial Repo",ClasificadosOnline Cupey Bajo de SKY REALTY (zona 2),Comercial * Cupey Bajo * REO,,"$285,000","Barrio-Cupey Bajo , San Juan - Río Piedras",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4802915
2,", Casa",ClasificadosOnline Los Altos De Ciudad Jardin de Palma Real Property Group,URB CIUDAD JARDIN-LOS ALTOS-1006 M2,4 Cuartos | 3 Baños,"$638,000","Urbanizacion-Los Altos De Ciudad Jardin , Gurabo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4816780
3,", Solar",ClasificadosOnline Marin de Dream Properties LLC,Terreno de 20 cuerdas frente al Lago ...,,"$175,000","Barrio-Marin , Patillas",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4817751
4,", Casa",ClasificadosOnline Sonadora de RepoExperts,"SONADORA , AGUAS BUENAS MANSION",>=6 Cuartos | 5 Baños,"$1,150,000","Barrio-Sonadora , Aguas Buenas",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4769002
5,", Casa",ClasificadosOnline Estancias Reales de Arrieta Realty Group,Urb. Estancias Reales Guaynabo,4 Cuartos | 2 Baños,"$1,095,000","Urbanizacion-Estancias Reales , Guaynabo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4814485
6,", Apartamento",ClasificadosOnline La Costa de Edgar Molinary,LA COSTA GARDEN APARTMENTS,3 Cuartos | 2 Baños,"$187,000","Condominio-La Costa , Fajardo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4805732
7,", Apartamento",ClasificadosOnline Balcones De San Juan de 1RUIZ REALTY GROUP,BELLA VISTA APT 3 H Y 2 B 2 PISCINA,3 Cuartos | 2 Baños,"$194,500","Condominio-Balcones De San Juan , San Juan - Río Piedras",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4815230
8,", Casa",ClasificadosOnline Royal Town de Dream Realty,PROPIEDAD (multi-uso),>=6 Cuartos | 3 Baños,"$975,000","Urbanizacion-Royal Town , Bayamón",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4645731
9,", Casa",ClasificadosOnline Llanos Costa de Unlimited Realty Group,Llanos Costa,3 Cuartos | 2 Baños,"$245,000","Barrio-Llanos Costa , Cabo Rojo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4814244


In [14]:
last_modified = driver.execute_script("return document.lastModified;")

In [15]:
last_modified

'08/28/2024 21:32:27'

In [16]:
import requests

# Send a GET request to the webpage
response = requests.get("https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4774562")  # Replace this URL with the URL of your webpage

# Check the Last-Modified header in the response
last_modified_header = response.headers.get("Last-Modified")

# Print the Last-Modified header
print("Last modified (from HTTP header):", last_modified_header)

Last modified (from HTTP header): None


In [ ]:
import re
import pandas as pd

def clean_real_estate_data(df):
    # Clean the 'Type' column by removing leading commas and whitespace
    df['Type'] = df['Type'].str.replace(r'^,\s*', '', regex=True)

    # Clean the 'Broker' column using the combined logic
    def clean_broker(broker_name):
        try:
            # Remove any text starting with " - " followed by anything after it
            broker_name = re.sub(r'\s*-\s*.*$', '', broker_name)
            
            # Remove any text starting with "#" followed by digits
            broker_name = re.sub(r'\s*#\d+', '', broker_name)
            
            # Remove any text starting with "lic." followed by digits
            broker_name = re.sub(r'\s*lic\.\s*\d+', '', broker_name)
            
        except Exception as e:
            print(f"An error occurred: {e}")

        # Return the cleaned broker name
        return broker_name.strip()
    
    # Apply the clean_broker function to the 'Broker' column
    df['Broker'] = df['Broker'].apply(clean_broker)

    # Drop the 'Title' column if needed
    # df.drop(columns=['Title'], inplace=True)

    # Extract the number of bedrooms and bathrooms from the 'Rooms' column
    df[['Bedrooms', 'Bathrooms']] = df['Rooms'].str.extract(r'(\d+)\s+Cuartos.*\|\s*(\d+)\s+Baños', expand=True)
    df[['Bedrooms', 'Bathrooms']] = df[['Bedrooms', 'Bathrooms']].fillna(0)

    # Split the 'Location' column by commas
    split_location = df['Location'].str.split


In [41]:
cleaned_df = clean_real_estate_data(df)

In [42]:
cleaned_df

,Type,Broker,Title,Rooms,Price,Location,Link,Bedrooms,Bathrooms,Neighborhood,City
0,Solar,House Hunt Real Estate #18152,BO. CACAO,,"$57,999","Barrio-Cacao , Quebradillas",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4767842,0,0,Barrio-Cacao,Quebradillas
1,Comercial Repo,SKY REALTY (zona 2),Comercial * Cupey Bajo * REO,,"$285,000","Barrio-Cupey Bajo , San Juan - Río Piedras",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4802915,0,0,Barrio-Cupey Bajo,San Juan - Río Piedras
2,Casa,Palma Real Property Group,URB CIUDAD JARDIN-LOS ALTOS-1006 M2,4 Cuartos | 3 Baños,"$638,000","Urbanizacion-Los Altos De Ciudad Jardin , Gurabo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4816780,4,3,Urbanizacion-Los Altos De Ciudad Jardin,Gurabo
3,Solar,Dream Properties LLC,Terreno de 20 cuerdas frente al Lago ...,,"$175,000","Barrio-Marin , Patillas",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4817751,0,0,Barrio-Marin,Patillas
4,Casa,RepoExperts,"SONADORA , AGUAS BUENAS MANSION",>=6 Cuartos | 5 Baños,"$1,150,000","Barrio-Sonadora , Aguas Buenas",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4769002,6,5,Barrio-Sonadora,Aguas Buenas
5,Casa,Arrieta Realty Group,Urb. Estancias Reales Guaynabo,4 Cuartos | 2 Baños,"$1,095,000","Urbanizacion-Estancias Reales , Guaynabo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4814485,4,2,Urbanizacion-Estancias Reales,Guaynabo
6,Apartamento,Edgar Molinary,LA COSTA GARDEN APARTMENTS,3 Cuartos | 2 Baños,"$187,000","Condominio-La Costa , Fajardo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4805732,3,2,Condominio-La Costa,Fajardo
7,Apartamento,1RUIZ REALTY GROUP,BELLA VISTA APT 3 H Y 2 B 2 PISCINA,3 Cuartos | 2 Baños,"$194,500","Condominio-Balcones De San Juan , San Juan - Río Piedras",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4815230,3,2,Condominio-Balcones De San Juan,San Juan - Río Piedras
8,Casa,Dream Realty,PROPIEDAD (multi-uso),>=6 Cuartos | 3 Baños,"$975,000","Urbanizacion-Royal Town , Bayamón",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4645731,6,3,Urbanizacion-Royal Town,Bayamón
9,Casa,Unlimited Realty Group,Llanos Costa,3 Cuartos | 2 Baños,"$245,000","Barrio-Llanos Costa , Cabo Rojo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4814244,3,2,Barrio-Llanos Costa,Cabo Rojo


In [43]:
cleaned_df.to_csv('clasificados_sample.csv',index=False)